<left>
    <img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Logo_SYGNET.png" width="90" alt="cognitiveclass.ai logo">
</left>

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/2d/Tensorflow_logo.svg" width="200" alt="cognitiveclass.ai logo">
</center>




# A.I. Neural Training
## Part I - Architecture and Design of LSTM Network

Postprocessing the simulation results from ZSOIL for Neural Network training.

## Objectives

By conducting this thorough data analysis, we'll gain a deeper understanding of the dataset and the underlying physical processes. This will not only help in building a better NN model but also provide valuable insights into the soil subsidence phenomenon in our simulation. These insights can guide feature selection, inform model architecture decisions, and improve interpretability of the NN's results.

*   Data Science with Python
*   Statistics

<h3>Table of Contents</h3>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#III. Feature Engineering"><b>II. Feature Engineering</b></a></li>
        <li><a href="#- Derived and Cumulative Variables">- Correlation and Causality</a></li>
    </ul>
</div>

<hr>

In [1]:
!pip install statsmodels

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from statsmodels.stats.stattools import omni_normtest, jarque_bera
from scipy.stats import skew, kurtosis

print(scipy.__version__)

1.7.3


### 1. Importing TensorFlow

First we install TensorFlow for our operations

In [2]:
!pip install tensorflow==2.9.0
!pip install numpy==1.21.4

In [4]:
import time
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.integrate import dblquad

Loading the <i>AI_Dataset</i>

In [5]:
# Load and preprocess the data
training_data = pd.read_csv('AI_Dataset.csv')
print(training_data.columns.tolist())

['TIME', 'SF', 'PUSHOVER LABEL', 'PUSHOVER LAMBDA', 'PUSHOVER U-CTRL', 'ARC LENGTH STEP', 'ARC LENGTH U-NORM', 'ARC LENGTH LOAD FACTOR', 'NR', 'X', 'Y', 'Displacement-X', 'Displacement-Y', 'Rotation-Z', 'Total head-', 'Residual Force-X', 'Residual Force-Y', 'Residual Heat flux-Z', 'Solid-Velocity-X', 'Solid-Velocity-Y', 'Solid-Acceleration-X', 'Solid-Acceleration-Y', 'Unnamed: 23', 'TIME.1', 'SF.1', 'PUSHOVER LABEL.1', 'PUSHOVER LAMBDA.1', 'PUSHOVER U-CTRL.1', 'ARC LENGTH STEP.1', 'ARC LENGTH U-NORM.1', 'ARC LENGTH LOAD FACTOR.1', 'ELEM.', 'GP', 'Z', 'Eff.Stress-XY', 'Eff.Stress-XZ', 'Eff.Stress-YZ', 'Tot.Stress-XZ', 'Tot.Stress-YZ', 'Strain-XX', 'Strain-YY', 'Strain-XY', 'Strain-ZZ', 'Strain-XZ', 'Strain-YZ', 'Strain-11', 'Strain-33', 'Strain-J2^1/2', 'Stress level', 'Saturation', 'Fluid velocity-X', 'Fluid velocity-Y', 'Fluid velocity-ABS', 'Pc', 'Pore pressure', 'alpha*(S*pF+<dpF_undr>)', 'Temperature', 'Unnamed: 60', 'Gradient_DisplacementY_X', 'Gradient_StressYY_X', 'Gradient_Stre

### Calculating Subsidence with probability integral method (PIM) Method

In [ ]:
!pip install joblib

import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from joblib import Parallel, delayed

# Load the training data
training_data = pd.read_csv('AI_Dataset.csv')

# Pre-compute the cKDTree for efficient neighbor searching
coordinates = training_data[['X', 'Y']].values
tree = cKDTree(coordinates)

In [ ]:
# Simplified q_function
def q_function(x_idx, training_data):
    row = training_data.iloc[x_idx]
    return (
        row['Displacement-Y'] * 0.3 + 
        row['Eff.Stress-XY'] * 0.2 + 
        row['Fluid velocity-Y'] * 0.1 + 
        row['Saturation'] * 0.1 + 
        row['Pore pressure'] * 0.1 + 
        row['Gradient_DisplacementY_X'] * 0.05 + 
        row['Gradient_StressYY_X'] * 0.05 + 
        row['Gradient_StressYY_Y'] * 0.05 + 
        row['Normalized_DisplacementY'] * 0.05 + 
        row['Normalized_StressYY'] * 0.05 + 
        row['Displacement-Magnitude'] * 0.05 + 
        row['Cumulative-Displacement-Y'] * 0.05 + 
        row['StressYY_DisplacementY_Interaction'] * 0.05 + 
        row['DisplacementY_Rate'] * 0.05 + 
        row['StressYY_Rate'] * 0.05 + 
        row['Cumulative_StressYY'] * 0.05 + 
        row['Log_StressYY'] * 0.05 + 
        row['PCA_1'] * 0.05 + 
        row['PCA_2'] * 0.05 + 
        row['PCA_3'] * 0.05
    )

# Optimized subsidence calculation with limited neighborhood and simplified summation
def calculate_subsidence_pim(x_idx, training_data, n=2, neighborhood_radius=5):
    x, y = training_data.iloc[x_idx][['X', 'Y']].values
    neighbors_idx = tree.query_ball_point([x, y], neighborhood_radius)
    
    subsidence = sum(
        q_function(neighbor_idx, training_data) / 
        max(np.sqrt((x - training_data.iloc[neighbor_idx]['X'])**2 +
                    (y - training_data.iloc[neighbor_idx]['Y'])**2), 1e-6) ** n
        for neighbor_idx in neighbors_idx
    )
    
    return subsidence

Computational Variant 1

In [ ]:
# Apply subsidence calculation to the dataset in parallel
def apply_subsidence_pim_to_dataset(training_data, n=2, neighborhood_radius=5):
    results = Parallel(n_jobs=-1, backend='loky', prefer="threads")(
        delayed(calculate_subsidence_pim)(idx, training_data, n, neighborhood_radius) 
        for idx in range(len(training_data))
    )
    training_data['PIM_Subsidence'] = results
    return training_data

# Calculate subsidence using PIM and add to the dataset
training_data = apply_subsidence_pim_to_dataset(training_data)

# Save or use the updated training_data with the PIM_Subsidence column
training_data.to_csv('PIM_Subsidence.csv', index=False)

In [ ]:
# Save or use the updated training_data with the PIM_Subsidence column
training_data.to_csv('PIM_Subsidence.csv', index=False)

Computational Variant 2

In [ ]:
# Apply subsidence calculation to the dataset in parallel
def apply_subsidence_pim_to_dataset(training_data, n=2, neighborhood_radius=5):
    results = Parallel(n_jobs=-1)(
        delayed(calculate_subsidence_pim)(idx, training_data, n, neighborhood_radius) 
        for idx in range(len(training_data))
    )
    training_data['PIM_Subsidence'] = results
    return training_data

# Calculate subsidence using PIM and add to the dataset
training_data = apply_subsidence_pim_to_dataset(training_data)

# Save or use the updated training_data with the PIM_Subsidence column
training_data.to_csv('PIM_Subsidence.csv', index=False)

### LSTM Network

Importing Libraries

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Simpler Model for Artificial Subsidence

Generating Artificial Subsidence

In [ ]:
data = pd.read_csv('AI_Dataset.csv')

data['Subsidence'] = (data['Displacement-Y'] * 0.3 + 
                      data['Eff.Stress-XY'] * 0.2 + 
                      data['Fluid velocity-Y'] * 0.1 + 
                      data['Saturation'] * 0.1 + 
                      data['Pore pressure'] * 0.1 + 
                      data['Gradient_DisplacementY_X'] * 0.05 + 
                      data['Gradient_StressYY_X'] * 0.05 + 
                      data['Gradient_StressYY_Y'] * 0.05 + 
                      data['Normalized_DisplacementY'] * 0.05 + 
                      data['Normalized_StressYY'] * 0.05 + 
                      data['Displacement-Magnitude'] * 0.05 + 
                      data['Cumulative-Displacement-Y'] * 0.05 + 
                      data['StressYY_DisplacementY_Interaction'] * 0.05 + 
                      data['DisplacementY_Rate'] * 0.05 + 
                      data['StressYY_Rate'] * 0.05 + 
                      data['Cumulative_StressYY'] * 0.05 + 
                      data['Log_StressYY'] * 0.05 + 
                      data['PCA_1'] * 0.05 + 
                      data['PCA_2'] * 0.05 + 
                      data['PCA_3'] * 0.05)

Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
subsidence_scaler = StandardScaler()
data['Subsidence'] = subsidence_scaler.fit_transform(data[['Subsidence']])

# Define a mapping from string to numeric values
mapping = {'elastic': 0, 'plastic': 1, 'brittle': 2}

# Apply the mapping to the column
data['Stress_Regime'] = data['Stress_Regime'].map(mapping)

# Handle any missing or unmapped values
data['Stress_Regime'] = data['Stress_Regime'].fillna(-1)  # or any other default value


### Model for artificial Subsidence

In [ ]:
# Select both original and engineered features for the LSTM model
features = data[['Displacement-Y','Eff.Stress-XY', 'Fluid velocity-Y', 'Solid-Velocity-Y', 'Saturation', 'Pore pressure',
                 'Gradient_DisplacementY_X', 'Gradient_StressYY_X', 'Gradient_StressYY_Y', 
                 'Normalized_DisplacementY', 'Normalized_StressYY', 'Displacement-Magnitude', 
                 'Cumulative-Displacement-Y', 'StressYY_DisplacementY_Interaction', 'DisplacementY_Rate', 
                 'StressYY_Rate', 'Cumulative_StressYY', 'Log_StressYY', 'PCA_1', 'PCA_2', 'PCA_3'] + 
                 [col for col in data.columns if 'Stress_Regime' in col]]

# Target variable
target = data['Subsidence']

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features (necessary for LSTM to learn effectively)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Reshape input data to 3D for LSTM (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_valid = X_valid.reshape(X_valid.shape[0], 1, X_valid.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Reshape targets to match LSTM input requirements
y_train = y_train.values.reshape(-1, 1)
y_valid = y_valid.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)


In [ ]:
# Define the LSTM model
model = tf.keras.models.Sequential()

# Adding LSTM layers with dropout
model.add(tf.keras.layers.LSTM(units=256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=128, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.2))

# Add Dense layer for output prediction
model.add(tf.keras.layers.Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Summary of the model
model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), verbose=2)

In [ ]:
# Evaluate on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# Convert predictions back to the original scale if needed (e.g., if you inverse transform the target)
# y_pred_original = scaler.inverse_transform(y_pred)
# y_test_original = scaler.inverse_transform(y_test)

In [ ]:
# Save the trained model for future use
model.save('artificial_subsidence_lstm_model.h5')

### <b>Model for PIM Subsidence</b>

Importing <i>PIM_Subsidence</I>

In [7]:
# Load the dataset (assuming it's already cleaned and engineered as AI_Dataset.csv)
data = pd.read_csv('PIM_Subsidence.csv')

# Convert 'Stress Regime' categorical column to numerical using one-hot encoding
data = pd.get_dummies(data, columns=['Stress_Regime'])

Checking for NaNs and infinity values

In [8]:
print(data.isna().sum())
print(data.applymap(np.isinf).sum())

TIME                     0
SF                       0
PUSHOVER LABEL           0
PUSHOVER LAMBDA          0
PUSHOVER U-CTRL          0
                        ..
PCA_1                    0
PCA_2                    0
PCA_3                    0
PIM_Subsidence           0
Stress_Regime_elastic    0
Length: 75, dtype: int64
TIME                     0
SF                       0
PUSHOVER LABEL           0
PUSHOVER LAMBDA          0
PUSHOVER U-CTRL          0
                        ..
PCA_1                    0
PCA_2                    0
PCA_3                    0
PIM_Subsidence           0
Stress_Regime_elastic    0
Length: 75, dtype: int64


Normalizing <i>PIM_Subsidence</i> column

In [9]:
from sklearn.preprocessing import StandardScaler
subsidence_scaler = StandardScaler()
data['PIM_Subsidence'] = subsidence_scaler.fit_transform(data[['PIM_Subsidence']])

Training Data configuration

In [11]:
# Select both original and engineered features for the LSTM model
features = data[['Displacement-Y','Eff.Stress-XY', 'Fluid velocity-Y', 'Solid-Velocity-Y', 'Saturation', 'Pore pressure',
                 'Gradient_DisplacementY_X', 'Gradient_StressYY_X', 'Gradient_StressYY_Y', 
                 'Normalized_DisplacementY', 'Normalized_StressYY', 'Displacement-Magnitude', 
                 'Cumulative-Displacement-Y', 'StressYY_DisplacementY_Interaction', 'DisplacementY_Rate', 
                 'StressYY_Rate', 'Cumulative_StressYY', 'Log_StressYY', 'PCA_1', 'PCA_2', 'PCA_3'] + 
                 [col for col in data.columns if 'Stress_Regime' in col]]

# Target variable
target = data['PIM_Subsidence']

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features (necessary for LSTM to learn effectively)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Reshape input data to 3D for LSTM (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_valid = X_valid.reshape(X_valid.shape[0], 1, X_valid.shape[1])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Reshape targets to match LSTM input requirements
y_train = y_train.values.reshape(-1, 1)
y_valid = y_valid.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:19: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.


LSTM Model

If the model has been uploaded from saved file - do not execute, as it will create a fresh and untrained version!

In [15]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.model_selection import KFold
import numpy as np
from time import time  # Import the time function

# Define the LSTM model
model = tf.keras.models.Sequential()
# Adding LSTM layers with increased dropout
model.add(tf.keras.layers.LSTM(units=256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.Dropout(0.4))  # Increased dropout rate
model.add(tf.keras.layers.LSTM(units=128, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.4))  # Increased dropout rate
# Add Dense layer for output prediction
model.add(tf.keras.layers.Dense(units=1))
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Summary of the model
model.summary()

# Early Stopping with Patience
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Learning Rate Adjustment
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

# Model Checkpointing
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# TensorBoard
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

# Data Augmentation
def augment_data(X, y, noise_factor=0.05):
    noise = noise_factor * np.random.randn(*X.shape)
    X_augmented = X + noise
    return X_augmented, y

X_train_aug, y_train_aug = augment_data(X_train, y_train)

# Cross-validation function
def cross_validate_and_train(model, X, y, epochs, batch_size, callbacks):
    kf = KFold(n_splits=5)
    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]
        history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), 
                            epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=2)
    return history

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1, 256)            285696    
                                                                 
 dropout_2 (Dropout)         (None, 1, 256)            0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               197120    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 482,945
Trainable params: 482,945
Non-trainable params: 0
_________________________________________________________________


Running TensorBoard in Jupyter Notebook

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

Simulation Start

In [19]:
# Train the model with cross-validation
history = cross_validate_and_train(model, X_train, y_train, epochs=500, batch_size=32, 
                                   callbacks=[early_stopping, lr_reduction, checkpoint, tensorboard])

# Save the trained model for future use
model.save('PIM_subsidence_lstm_model.h5')

# Store predictions
lstm_train_predictions = model.predict(X_train).flatten()
lstm_test_predictions = model.predict(X_test).flatten()

# Optionally, you can also store validation predictions
lstm_valid_predictions = model.predict(X_valid).flatten()

# Create a DataFrame
predictions_df = pd.DataFrame({
    'Train Predictions': lstm_train_predictions,
    'Test Predictions': lstm_test_predictions,
    'Validation Predictions': lstm_valid_predictions
})

# Export to CSV
predictions_df.to_csv('predictions.csv', index=False)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.

Epoch 1/500
280/280 - 10s - loss: 0.0453 - val_loss: 0.0016 - lr: 1.0000e-05 - 10s/epoch - 37ms/step
Epoch 2/500
280/280 - 9s - loss: 0.0450 - val_loss: 0.0016 - lr: 1.0000e-05 - 9s/epoch - 31ms/step
Epoch 3/500
280/280 - 7s - loss: 0.0449 - val_loss: 0.0016 - lr: 1.0000e-05 - 7s/epoch - 27ms/step
Epoch 4/500
280/280 - 7s - loss: 0.0439 - val_loss: 0.0016 - lr: 1.0000e-05 - 7s/epoch - 26ms/step
Epoch 5/500
280/280 - 7s - loss: 0.0472 - val_loss: 0.0016 - lr: 1.0000e-05 - 7s/epoch - 26ms/step
Epoch 6/500
280/280 - 8s - loss: 0.0443 - val_loss: 0.0016 - lr: 1.0000e-05 - 8s/epoch - 27ms/step
Epoch 7/500
280/280 - 7s - loss: 0.0439 - val_loss: 0.0016 - lr: 1.0000e-05 - 7s/epoch - 26ms/step
Epoch 8/500
280/280 - 7s - loss: 0.0447 - val_loss: 0.0016 - lr: 1.0000e-05 - 7s/epoch - 26ms/step
Epoch 9/500
280/280 - 8s - loss: 0.0455 - val_loss: 0.0016 - lr: 1.0000e-05 - 8s/epoch - 27ms/step
Epoch 10/500
280/280 - 9s - loss: 0.0468 - val_loss: 0.0016 - lr: 1.0000e-05 - 9s/epoch - 31ms/step
Epoch 1

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


280/280 - 7s - loss: 0.0315 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 2/500
280/280 - 7s - loss: 0.0265 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 3/500
280/280 - 7s - loss: 0.0291 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 4/500
280/280 - 7s - loss: 0.0291 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 5/500
280/280 - 7s - loss: 0.0282 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 6/500
280/280 - 8s - loss: 0.0295 - val_loss: 0.0774 - lr: 1.0000e-05 - 8s/epoch - 30ms/step
Epoch 7/500
280/280 - 7s - loss: 0.0306 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 26ms/step
Epoch 8/500
280/280 - 7s - loss: 0.0282 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 9/500
280/280 - 7s - loss: 0.0310 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 24ms/step
Epoch 10/500
280/280 - 7s - loss: 0.0265 - val_loss: 0.0774 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 11/500
280/280 

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


...

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


280/280 - 7s - loss: 0.0453 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 2/500
280/280 - 7s - loss: 0.0457 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 3/500
280/280 - 7s - loss: 0.0447 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 4/500
280/280 - 7s - loss: 0.0456 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 24ms/step
Epoch 5/500
280/280 - 7s - loss: 0.0508 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 24ms/step
Epoch 6/500
280/280 - 7s - loss: 0.0453 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 7/500
280/280 - 7s - loss: 0.0467 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 8/500
280/280 - 7s - loss: 0.0456 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 9/500
280/280 - 7s - loss: 0.0455 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 10/500
280/280 - 7s - loss: 0.0448 - val_loss: 0.0020 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 11/500
280/280 

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:113: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_mask = np.zeros(_num_samples(X), dtype=np.bool)


280/280 - 7s - loss: 0.0280 - val_loss: 0.0780 - lr: 1.0000e-05 - 7s/epoch - 26ms/step
Epoch 2/500
280/280 - 7s - loss: 0.0266 - val_loss: 0.0780 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 3/500
280/280 - 7s - loss: 0.0279 - val_loss: 0.0780 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 4/500
280/280 - 7s - loss: 0.0301 - val_loss: 0.0780 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 5/500
280/280 - 7s - loss: 0.0292 - val_loss: 0.0780 - lr: 1.0000e-05 - 7s/epoch - 24ms/step
Epoch 6/500
280/280 - 7s - loss: 0.0268 - val_loss: 0.0781 - lr: 1.0000e-05 - 7s/epoch - 24ms/step
Epoch 7/500
280/280 - 7s - loss: 0.0247 - val_loss: 0.0781 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 8/500
280/280 - 7s - loss: 0.0321 - val_loss: 0.0781 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 9/500
280/280 - 7s - loss: 0.0270 - val_loss: 0.0781 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 10/500
280/280 - 7s - loss: 0.0269 - val_loss: 0.0781 - lr: 1.0000e-05 - 7s/epoch - 25ms/step
Epoch 11/500
280/280 

ValueError: All arrays must be of the same length

In [24]:
# Create a DataFrame
train_predictions_df = pd.DataFrame({'Train Predictions': lstm_train_predictions})
test_predictions_df = pd.DataFrame({'Test Predictions': lstm_test_predictions})
validation_predictions_df = pd.DataFrame({'Validation Predictions': lstm_valid_predictions})
    


# Export to CSV
train_predictions_df.to_csv('train_predictions.csv', index=False)
test_predictions_df.to_csv('test_predictions.csv', index=False)
validation_predictions_df.to_csv('validation_predictions.csv', index=False)

Results Evaluation

In [ ]:
# Evaluate on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Predictions on the test set
y_pred = model.predict(X_test)

Save LSTM model for future use

In [ ]:
# Save the trained model for future use
model.save('PIM_subsidence_lstm_model.h5')

Upload the saved model

In [18]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('PIM_subsidence_lstm_model.h5')

## Neural Network Visualisation

### With <i>TensorFlow/Keras Model Plotting</i>:

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### With <i>PlotNeuralNet</i>

Install LaTex

Checking LaTex
$$
f(x) = \int_{-\infty}^{\infty} e^{-x^2} dx
$$

Clone PlotNeuralNet Repository:

In [ ]:
!git clone https://github.com/HarisIqbal88/PlotNeuralNet.git
%cd PlotNeuralNet
!pwd

In [ ]:
# Navigate to the PlotNeuralNet directory
%cd /resources/DL0120EN/labs/Week6/PlotNeuralNet

# Add the PlotNeuralNet directory to Python path
import sys
sys.path.append('/resources/DL0120EN/labs/Week6/PlotNeuralNet')

# Import the necessary modules
from pycore.tikzeng import *
from pycore.blocks import *

<b> Create Your LSTM Diagram:</b>

Modify the example.py script to match your LSTM model's architecture. You would need to tailor the blocks (e.g., ConvConvRelu, Pool, LSTM) to match the layers in your model. For an LSTM, you may use custom blocks that represent the LSTM cells.

In [ ]:
# Define your architecture
arch = [
    to_head('..'),
    to_cor(),
    to_begin(),

    to_Conv("conv1", 512, 64, offset="(0,0,0)", to="(0,0,0)", height=64, depth=64, width=2),
    to_Pool("pool1", offset="(0,0,0)", to="(conv1-east)"),
    to_Conv("conv2", 128, 64, offset="(1,0,0)", to="(pool1-east)", height=32, depth=32, width=2),
    to_connection("pool1", "conv2"),
    to_Pool("pool2", offset="(0,0,0)", to="(conv2-east)", height=28, depth=28, width=1),
    to_SoftMax("soft1", 10, "(3,0,0)", "(pool2-east)"),
    to_end()
]

def main():
    namefile = 'lstm_example'
    to_generate(arch, namefile + '.tex')

if __name__ == '__main__':
    main()
    

Generate and Render the Diagram:

You can use the `IFrame` tool to convert the `.tex` file to a PDF and then display it in your notebook:

In [ ]:
import os
from IPython.display import IFrame

os.system("pdflatex lstm_example.tex")
IFrame("lstm_example.pdf", width=600, height=400)

You can use the `pymupdf` library to convert the PDF to a JPEG image:

In [ ]:
!pip install pymupdf

import fitz  # PyMuPDF
from PIL import Image
from io import BytesIO
import IPython.display as display

# Open the PDF file
pdf_document = fitz.open("lstm_example.pdf")

# Select the first page
page = pdf_document.load_page(0)

# Convert the page to a pixmap (image)
pix = page.get_pixmap()

# Convert the pixmap to a PIL Image
img = Image.open(BytesIO(pix.tobytes()))

# Display the image in the Jupyter Notebook
display.display(img)

Compile the .tex file using LaTeX:

In [ ]:
import os
from IPython.display import IFrame
from pdf2image import convert_from_path

# Convert .tex to .pdf
os.system("pdflatex lstm_example.tex")

# Display the PDF
display(IFrame("lstm_example.pdf", width=600, height=400))

# Convert PDF to JPEG
pages = convert_from_path('lstm_example.pdf', 300)
for page in pages:
    page.save('lstm_example.jpg', 'JPEG')

# Display the JPEG
from IPython.display import Image
Image(filename='lstm_example.jpg')

This will produce a PDF that visually represents your LSTM model. You can use this PDF in your reports or presentations.

Display the generated PDF in your notebook:

In [ ]:
from IPython.display import IFrame
IFrame("lstm_example.pdf", width=600, height=400)